# Load and Explore Data

Load and explore the data

In [16]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Fetch data

In [17]:
# Historical data from Github
url = 'https://raw.githubusercontent.com/ijyliu/mcbroken-daily-historical/refs/heads/main/mcbroken_daily_most_recent_on_20250215.csv'

# Read the data
historical_data = pd.read_csv(url)
historical_data

# Drop entry for 2025-02-15 - this will come from the more recent S3 data
historical_data = historical_data[historical_data['date'] != '2025-02-15']

In [18]:
# More recent data from S3
s3_url = 'https://mcbroken-bucket.s3.us-west-1.amazonaws.com/updated-mcbroken.csv'

# Read the data
recent_data = pd.read_csv(s3_url)
recent_data

,total_machines,date,datetime,broken_machines
0,11867,2025-02-16,2025-02-16T19:40:17.382296,1283
1,11773,2025-02-15,2025-02-15T19:40:17.269596,1107


## Stack data

In [19]:
lim_hist = historical_data[['date', 'broken_machines', 'total_machines']]
lim_recent = recent_data[['date', 'broken_machines', 'total_machines']]
df = pd.concat([lim_hist, lim_recent], axis=0).sort_values('date').reset_index(drop=True)
# Rename broken_machines to "Broken Machines" and total_machines to "Total Machines", and date to "Date"
df = df.rename(columns={'broken_machines': 'Broken Machines', 'total_machines': 'Total Machines', 'date': 'Date'})
# Add percent broken
df['Percent Broken'] = df['Broken Machines'] / df['Total Machines'] * 100
# Daily revenue losses at $625 per machine per day
# https://www.dailymail.co.uk/sciencetech/article-14019935/McDonalds-fans-rejoice-company-wins-legal-battle-fix-one-lifes-greatest-nuisances.html
df['Revenue Losses'] = df['Broken Machines'] * 625
df

,Date,Broken Machines,Total Machines,Percent Broken,Revenue Losses
0,2020-10-25,354,6818,5.192138,221250
1,2020-10-26,373,6844,5.450029,233125
2,2020-10-27,427,7488,5.702457,266875
3,2020-10-28,391,8294,4.714251,244375
4,2020-10-29,374,7461,5.012733,233750
...,...,...,...,...,...
1442,2025-02-12,1277,11636,10.974562,798125
1443,2025-02-13,1189,11707,10.156317,743125
1444,2025-02-14,1068,11428,9.345467,667500
1445,2025-02-15,1107,11773,9.402871,691875


# Add entries for missing days

In [20]:
# Create df of all days between min and max dates
min_date = df['Date'].min()
max_date = df['Date'].max()
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')

# Merge with df
# Convert type to datetime
df['Date'] = pd.to_datetime(df['Date'])
all_dates_df = pd.DataFrame(all_dates, columns=['Date'])
df = pd.merge(all_dates_df, df, on='Date', how='left').sort_values('Date').reset_index(drop=True)
df

,Date,Broken Machines,Total Machines,Percent Broken,Revenue Losses
0,2020-10-25,354.0,6818.0,5.192138,221250.0
1,2020-10-26,373.0,6844.0,5.450029,233125.0
2,2020-10-27,427.0,7488.0,5.702457,266875.0
3,2020-10-28,391.0,8294.0,4.714251,244375.0
4,2020-10-29,374.0,7461.0,5.012733,233750.0
...,...,...,...,...,...
1571,2025-02-12,1277.0,11636.0,10.974562,798125.0
1572,2025-02-13,1189.0,11707.0,10.156317,743125.0
1573,2025-02-14,1068.0,11428.0,9.345467,667500.0
1574,2025-02-15,1107.0,11773.0,9.402871,691875.0


## Plot time series in plotly

In [21]:
# Plot using Plotly Express
fig = px.line(df, x='Date', y=['Broken Machines', 'Total Machines'],
              title='Machines Over Time',
              labels={'value': 'Number of Machines', 'variable': 'Machine Status'},
              line_shape='linear')

# Center title
fig.update_layout(title_x=0.5)

# Show the plot
fig.show()

In [22]:
# Just broken machines
fig = px.line(df, x='Date', y='Broken Machines',
              title='Broken Machines Over Time',
              labels={'Broken Machines': 'Number of Broken Machines'},
              line_shape='linear')

# Center title
fig.update_layout(title_x=0.5)

# Show the plot
fig.show()

In [23]:
# Percent broken
fig = px.line(df, x='Date', y='Percent Broken',
              title='Percent Broken Machines Over Time',
              labels={'Percent Broken': 'Percent of Broken Machines'},
              line_shape='linear')

# Center title
fig.update_layout(title_x=0.5)

# Show the plot
fig.show()

In [24]:
# Percent broken
fig = px.line(df, x='Date', y='Revenue Losses',
              title='Revenue Losses From Broken Machines ($625 per day per machine)',
              labels={'Revenue Losses': 'Revenue Losses'},
              line_shape='linear')

# Center title
fig.update_layout(title_x=0.5)

# Dollar format
fig.update_layout(yaxis_tickprefix='$')

# Show the plot
fig.show()

## Outliers

In [25]:
# Mark outliers
# December 14, 2020
# July 1, 2021 - Jan 1, 2023, any day with under 12,500 machines
# Jan 2, 2023 - July 1, 2024, any day with under 7,500 machines
# November 15, 2024
df['Outlier'] = False
df.loc[df['Date'] == '2020-12-14', 'Outlier'] = True
df.loc[(df['Date'] >= '2021-07-01') & (df['Date'] <= '2023-01-01') & (df['Total Machines'] < 12500), 'Outlier'] = True
df.loc[(df['Date'] >= '2023-01-02') & (df['Date'] <= '2024-07-01') & (df['Total Machines'] < 7500), 'Outlier'] = True
df.loc[df['Date'] == '2024-11-15', 'Outlier'] = True
# August 12 to September 9, 2024
df.loc[(df['Date'] >= '2024-08-12') & (df['Date'] <= '2024-09-09'), 'Outlier'] = True
# Below 1,000 broken machines december 1, 2023 to june 1, 2024
df.loc[(df['Date'] >= '2023-12-01') & (df['Date'] <= '2024-06-01') & (df['Broken Machines'] < 1000), 'Outlier'] = True
# Above 2,500 broken machines december 1, 2023 to june 1, 2024
df.loc[(df['Date'] >= '2023-12-01') & (df['Date'] <= '2024-06-01') & (df['Broken Machines'] > 2500), 'Outlier'] = True
# Print outliers
df[df['Outlier']]

,Date,Broken Machines,Total Machines,Percent Broken,Revenue Losses,Outlier
50,2020-12-14,395.0,5839.0,6.764857,246875.0,True
255,2021-07-07,127.0,2276.0,5.579965,79375.0,True
256,2021-07-08,153.0,2586.0,5.916473,95625.0,True
257,2021-07-09,144.0,2329.0,6.182911,90000.0,True
258,2021-07-10,170.0,2948.0,5.766621,106250.0,True
...,...,...,...,...,...,...
1412,2024-09-06,1985.0,14895.0,13.326620,1240625.0,True
1413,2024-09-07,1894.0,14895.0,12.715676,1183750.0,True
1414,2024-09-08,2167.0,15023.0,14.424549,1354375.0,True
1415,2024-09-09,2128.0,14806.0,14.372552,1330000.0,True


In [26]:
# Create the figure
fig = go.Figure()

# Add the line trace
fig.add_trace(go.Scatter(x=df['Date'], y=df['Total Machines'], mode='lines', name='Total Machines'))

# Iterate through the data and add a shape for each outlier day
for i in range(len(df)):
    if df['Outlier'][i]:
        # Shade the *entire* day.  Important for time series!
        date_fmt = pd.to_datetime(df['Date'][i])
        start_date = date_fmt.strftime('%Y-%m-%d 00:00:00') # Start of the day
        end_date = date_fmt.strftime('%Y-%m-%d 23:59:59')   # End of the day
        fig.add_shape(
            type="rect",
            x0=start_date,  # Start of the day
            x1=end_date,    # End of the day
            y0=0,
            y1=20000,
            line=dict(width=0),  # No border
            fillcolor="rgba(255, 0, 0, 0.2)",  # Red with 20% opacity
            layer="below"  # Place the rectangle behind the line
        )


# Improve layout (optional)
fig.update_layout(
    title="Line Chart with Outlier Shading",
    xaxis_title="Date",
    yaxis_title="Value",
    xaxis_range=[min(pd.to_datetime(df['Date'])) - pd.Timedelta(days=1), max(pd.to_datetime(df['Date'])) + pd.Timedelta(days=1)] # Add some padding to x-axis
)

fig.show()



In [27]:
# Create the figure
fig = go.Figure()

# Add the line trace
fig.add_trace(go.Scatter(x=df['Date'], y=df['Broken Machines'], mode='lines', name='Broken Machines'))

# Iterate through the data and add a shape for each outlier day
for i in range(len(df)):
    if df['Outlier'][i]:
        # Shade the *entire* day.  Important for time series!
        date_fmt = pd.to_datetime(df['Date'][i])
        start_date = date_fmt.strftime('%Y-%m-%d 00:00:00') # Start of the day
        end_date = date_fmt.strftime('%Y-%m-%d 23:59:59')   # End of the day
        fig.add_shape(
            type="rect",
            x0=start_date,  # Start of the day
            x1=end_date,    # End of the day
            y0=0,
            y1=5000,
            line=dict(width=0),  # No border
            fillcolor="rgba(255, 0, 0, 0.2)",  # Red with 20% opacity
            layer="below"  # Place the rectangle behind the line
        )


# Improve layout (optional)
fig.update_layout(
    title="Line Chart with Outlier Shading",
    xaxis_title="Date",
    yaxis_title="Value",
    xaxis_range=[min(pd.to_datetime(df['Date'])) - pd.Timedelta(days=1), max(pd.to_datetime(df['Date'])) + pd.Timedelta(days=1)] # Add some padding to x-axis
)

fig.show()


In [28]:
# Zooming in on the past year
df_past_yr = df.tail(365)

# Percent broken
fig = px.line(df_past_yr, x='Date', y='Revenue Losses',
              title='Revenue Losses From Broken Machines ($625 per day per machine)',
              labels={'Revenue Losses': 'Revenue Losses'},
              line_shape='linear')

# Center title
fig.update_layout(title_x=0.5)

# Dollar format
fig.update_layout(yaxis_tickprefix='$')

# Show the plot
fig.show()

## Add flags for train vs test sets

In [29]:
df['Train'] = True
# We will be making 30-day ahead forecasts, so last 30 days should be test
df.loc[df['Date'] > df['Date'].max() - pd.Timedelta(days=30), 'Train'] = False
# Print date ranges by Train
df.groupby('Train')['Date'].agg(['min', 'max'])

,min,max
Train,,
False,2025-01-18,2025-02-16
True,2020-10-25,2025-01-17


## Save to data folder

In [30]:
# save to excel in Data/Clean_McBroken_Daily.xlsx
df.to_excel('Data/Clean_McBroken_Daily.xlsx', index=False)